<a href="https://colab.research.google.com/github/JorgeAccardi/auscultacion-presa/blob/main/An%C3%A1lisis_de_Inclin%C3%B3metros_(GKN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#INCLINOMETROS

#Análisis de datos de archivos GKN

##Carga de datos, procesamiento y generación del DATAFRAME

In [1]:
import pandas as pd
import os

def procesar_gkn_inclinometros(file_path):
    try:
        print(f"Procesando el archivo: {file_path}")
        with open(file_path, 'r') as file:
            lines = file.readlines()
            fecha = next((line.split(":")[1].strip() for line in lines if line.startswith("DATE")), None)
            nro_sonda = next((line.split(":")[1].strip() for line in lines if line.startswith("PROBE NO.")), None)
            hole_no = next((line.split(":")[1].strip() for line in lines if line.startswith("HOLE NO.")), None)
            start_idx = next((i + 2 for i, line in enumerate(lines) if '#READINGS:' in line), None)
            if start_idx is None:
                raise ValueError(f"No se encontraron lecturas en el archivo {file_path}")
            data = [line.strip().split(',') for line in lines[start_idx:]]
            df = pd.DataFrame(data, columns=['Profundidad', 'A+', 'A-', 'B+', 'B-'])
            df = df.apply(pd.to_numeric, errors='coerce')
            df['Fecha'] = pd.to_datetime(fecha, dayfirst=True, errors='coerce').strftime('%d/%m/%Y')
            df['Inclinómetro'] = hole_no
            df['Nro. Sonda'] = nro_sonda
            df = df[['Fecha', 'Inclinómetro', 'Profundidad', 'A+', 'A-', 'B+', 'B-', 'Nro. Sonda']]
            print(f"Archivo procesado correctamente: {file_path}")
            return df
    except Exception as e:
        print(f"Error procesando el archivo {file_path}: {e}")
        return None

def procesar_varios_gkn(file_list):
    dfs = []
    for file_path in file_list:
        df = procesar_gkn_inclinometros(file_path)
        if df is not None:
            dfs.append(df)
    if dfs:
        df_final = pd.concat(dfs, ignore_index=True)
        print("Todos los archivos fueron procesados correctamente.")
        return df_final
    else:
        print("No se pudieron procesar los archivos.")
        return None

In [2]:
import os
from google.colab import files

def seleccionar_fuente_gkn():
    print("Selecciona la fuente de tus archivos GKN:")
    print("1 - Subir ZIP con todos los archivos desde tu computadora")
    print("2 - Carpeta en Google Drive")
    print("3 - Carpeta en /content de Colab")
    print("4 - Seleccionar carpeta localmente (visual, solo en Colab)")
    fuente = input("Escribe 1, 2, 3 o 4 y presiona ENTER: ")
    return fuente.strip()

file_names = []
fuente = seleccionar_fuente_gkn()

if fuente == "1":
    print("Sube el archivo .zip que contiene todos los archivos GKN:")
    uploaded = files.upload()
    import zipfile
    for filename in uploaded.keys():
        if filename.endswith('.zip'):
            with zipfile.ZipFile(filename, 'r') as zip_ref:
                zip_ref.extractall('GKN_folder')
            print("ZIP descomprimido en /content/GKN_folder")
        else:
            print("El archivo subido no es un .zip")
    carpeta_gkn = '/content/GKN_folder'
    file_names = [os.path.join(carpeta_gkn, f) for f in os.listdir(carpeta_gkn) if f.lower().endswith('.gkn')]
    print("Archivos GKN encontrados:", file_names)

elif fuente == "2":
    from google.colab import drive
    drive.mount('/content/drive')
    carpeta_gkn = input("Pega la ruta completa de la carpeta en Google Drive (ej: /content/drive/MyDrive/MisGKN): ").strip()
    if not os.path.exists(carpeta_gkn):
        raise ValueError("La carpeta especificada no existe.")
    file_names = [os.path.join(carpeta_gkn, f) for f in os.listdir(carpeta_gkn) if f.lower().endswith('.gkn')]
    print("Archivos GKN encontrados:", file_names)

elif fuente == "3":
    carpeta_gkn = input("Pega la ruta de la carpeta en /content (ej: /content/GKN): ").strip()
    if not os.path.exists(carpeta_gkn):
        raise ValueError("La carpeta especificada no existe.")
    file_names = [os.path.join(carpeta_gkn, f) for f in os.listdir(carpeta_gkn) if f.lower().endswith('.gkn')]
    print("Archivos GKN encontrados:", file_names)

elif fuente == "4":
    try:
        from google.colab import widgets
        import shutil
        print("Selecciona la carpeta con los archivos GKN en tu computadora.")
        # Usar el selector visual
        folder_uploaded = files.upload()
        local_folder = '/content/selected_folder'
        os.makedirs(local_folder, exist_ok=True)
        # Mueve los archivos subidos a una carpeta
        for filename in folder_uploaded.keys():
            shutil.move(filename, os.path.join(local_folder, filename))
        file_names = [os.path.join(local_folder, f) for f in os.listdir(local_folder) if f.lower().endswith('.gkn')]
        print("Archivos GKN encontrados:", file_names)
    except Exception as e:
        print("Funcionalidad visual de carpeta local no disponible. Usa ZIP o Drive para carpetas masivas.")
        file_names = []

else:
    print("Opción no válida. Ejecuta la celda de nuevo y elige 1, 2, 3 o 4.")

# Procesar todos los archivos encontrados (si hay)
if file_names:
    df_gkn = procesar_varios_gkn(file_names)
    if df_gkn is not None:
        display(df_gkn.head())
else:
    print("No se encontraron archivos para procesar.")

Selecciona la fuente de tus archivos GKN:
1 - Subir ZIP con todos los archivos desde tu computadora
2 - Carpeta en Google Drive
3 - Carpeta en /content de Colab
4 - Seleccionar carpeta localmente (visual, solo en Colab)
Escribe 1, 2, 3 o 4 y presiona ENTER: 1
Sube el archivo .zip que contiene todos los archivos GKN:


Saving MD-IN4-D1.zip to MD-IN4-D1.zip
ZIP descomprimido en /content/GKN_folder
Archivos GKN encontrados: ['/content/GKN_folder/MD-IN4-D1_012.gkn', '/content/GKN_folder/MD-IN4-D1_062.gkn', '/content/GKN_folder/MD-IN4-D1_025.gkn', '/content/GKN_folder/MD-IN4-D1_002.gkn', '/content/GKN_folder/MD-IN4-D1_009.gkn', '/content/GKN_folder/MD-IN4-D1_094.gkn', '/content/GKN_folder/MD-IN4-D1_061.gkn', '/content/GKN_folder/MD-IN4-D1_101.gkn', '/content/GKN_folder/MD-IN4-D1_036.gkn', '/content/GKN_folder/MD-IN4-D1_065.gkn', '/content/GKN_folder/MD-IN4-D1_057.gkn', '/content/GKN_folder/MD-IN4-D1_018.gkn', '/content/GKN_folder/MD-IN4-D1_091.gkn', '/content/GKN_folder/MD-IN4-D1_015.gkn', '/content/GKN_folder/MD-IN4-D1_074.gkn', '/content/GKN_folder/MD-IN4-D1_023.gkn', '/content/GKN_folder/MD-IN4-D1_066.gkn', '/content/GKN_folder/MD-IN4-D1_042.gkn', '/content/GKN_folder/MD-IN4-D1_116.gkn', '/content/GKN_folder/MD-IN4-D1_112.gkn', '/content/GKN_folder/MD-IN4-D1_090.gkn', '/content/GKN_folder/MD-IN4-D1_08

,Fecha,Inclinómetro,Profundidad,A+,A-,B+,B-,Nro. Sonda
0,09/12/2020,MD-IN4-D1,87.0,269.0,-272.0,160.0,-141.0,Probe-93
1,09/12/2020,MD-IN4-D1,86.5,277.0,-281.0,117.0,-80.0,Probe-93
2,09/12/2020,MD-IN4-D1,86.0,256.0,-263.0,43.0,-6.0,Probe-93
3,09/12/2020,MD-IN4-D1,85.5,202.0,-206.0,-13.0,50.0,Probe-93
4,09/12/2020,MD-IN4-D1,85.0,182.0,-188.0,-89.0,124.0,Probe-93


In [3]:
df_gkn

,Fecha,Inclinómetro,Profundidad,A+,A-,B+,B-,Nro. Sonda
0,09/12/2020,MD-IN4-D1,87.0,269.0,-272.0,160.0,-141.0,Probe-93
1,09/12/2020,MD-IN4-D1,86.5,277.0,-281.0,117.0,-80.0,Probe-93
2,09/12/2020,MD-IN4-D1,86.0,256.0,-263.0,43.0,-6.0,Probe-93
3,09/12/2020,MD-IN4-D1,85.5,202.0,-206.0,-13.0,50.0,Probe-93
4,09/12/2020,MD-IN4-D1,85.0,182.0,-188.0,-89.0,124.0,Probe-93
...,...,...,...,...,...,...,...,...
20876,02/09/2023,MD-IN4-D1,2.5,212.0,-222.0,-34.0,65.0,PROB 74
20877,02/09/2023,MD-IN4-D1,2.0,138.0,-145.0,75.0,-47.0,PROB 74
20878,02/09/2023,MD-IN4-D1,1.5,75.0,-84.0,186.0,-154.0,PROB 74
20879,02/09/2023,MD-IN4-D1,1.0,182.0,-193.0,259.0,-229.0,PROB 74
